In [4]:
import sys
import torch
print(f"Python version: {sys.version}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")


Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
CUDA available: True
CUDA version: 12.4


In [5]:
# Check Python version
import sys
print(f"Python version: {sys.version}")

# Install system dependencies
!apt-get update -qq
!apt-get install -y libreoffice poppler-utils tesseract-ocr -qq  # For DOCX to PDF, pdf2image, and OCR

# Clean up existing torch-related packages to avoid conflicts
#!pip uninstall torch torchaudio torchvision fastai layoutparser detectron2 fvcore -y --quiet

# Install latest Python dependencies
!pip install torch torchvision pdf2image opencv-python layoutparser pytesseract --quiet

# Install prerequisites for detectron2
!pip install cython pycocotools fvcore --quiet

# Install detectron2 from source (compatible with torch 2.6.0+cu124)
#!pip install 'git+https://github.com/facebookresearch/detectron2.git'

!pip install python-docx
!pip install --upgrade pillow
!sudo apt-get install tesseract-ocr-eng
#!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'
!pip install "layoutparser[ocr]"

!pip install sentence-transformers faiss-cpu  fastapi uvicorn pyngrok
!pip install faiss-gpu

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
  Cloning https://github.com/facebookresearch/detectron2.git (to revision ff53992b1985b63bd3262b5a36167098e3dada02) to /tmp/pip-req-build-1f41g4s5
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-1f41g4s5
  Running command git rev-parse -q --verify 'sha^ff53992b1985b63bd3262b5a36167098e3dada02'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git ff53992b1985b63bd3262b5a36167098e3dada02
  Running co

In [6]:
import torch
import torchvision
import layoutparser as lp
import detectron2
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.model_zoo import model_zoo
import os
import subprocess
import json
import traceback
from datetime import datetime
from pathlib import Path
from pdf2image import convert_from_path
import cv2
import pytesseract
import numpy as np
from PIL import Image
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import re
from sentence_transformers import SentenceTransformer
import faiss
import psycopg2
from psycopg2.extras import execute_values
from google.colab import drive

# Configuration flag
USE_POSTGRES = True

# Neon PostgreSQL connection details
DB_CONFIG = {
    "dbname": "neondb",
    "user": "neondb_owner",
    "password": "npg_DU3Vxoi6cCIu",
    "host": "ep-purple-sound-a4z952yb-pooler.us-east-1.aws.neon.tech",
    "port": "5432",
    "sslmode": "require"
}

# Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Mount Google Drive
drive.mount('/content/drive')

# Set up directories
BASE_DIR = "/content/drive/My Drive/lifesciences"
SOURCE_DIR = Path(BASE_DIR) / "training_documents"
PDF_STAGING_DIR = Path(BASE_DIR) / "staging/pdf"
IMG_STAGING_DIR = Path(BASE_DIR) / "post_label/images"
ANNOTATED_DIR = Path(BASE_DIR) / "staging/annotated"
MODELS_DIR = Path(BASE_DIR) / "models"
RETRAINED_STAGING_DIR = Path(BASE_DIR) / "retrained_staging"
RETRAINED_ANNOTATED_DIR = RETRAINED_STAGING_DIR / "annotated"
FAISS_INDEX_PATH = RETRAINED_STAGING_DIR / "faiss_index.index"
RETRAINED_MODELS_DIR = RETRAINED_STAGING_DIR / "models"
COCO_JSON_PATH = Path(BASE_DIR) / "post_label/result.json"

# Create directories
for directory in [SOURCE_DIR, PDF_STAGING_DIR, IMG_STAGING_DIR, ANNOTATED_DIR, MODELS_DIR, RETRAINED_STAGING_DIR, RETRAINED_ANNOTATED_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

Image.MAX_IMAGE_PIXELS = 100000000

# Global variables
id_seq = 1
sentence_model = None
faiss_index = None
model = None
ocr_agent = None
db_conn = None
db_cursor = None
run_number = 1
run_date_time = datetime.now().isoformat()
logged_in_user = "admin"

COLORS = {
    "title": (255, 0, 0), "text": (0, 255, 0), "list": (0, 0, 255),
    "table": (255, 255, 0), "figure": (255, 0, 255)
}

LABEL_MAP = {0: "figure", 1: "list", 2: "table", 3: "text", 4: "title"}

def map_and_update_coco_paths(json_path, image_dir):
    try:
        with open(json_path, "r") as f:
            coco_data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON {json_path}: {e}")
        raise

    actual_images = {f: f for f in os.listdir(image_dir) if f.endswith(".png")}
    print(f"Actual images in {image_dir}: {list(actual_images.keys())}")

    updated_images = []
    for image in coco_data["images"]:
        original_name = image["file_name"]
        cleaned_name = os.path.basename(original_name.replace("./images/", ""))
        if cleaned_name in actual_images:
            image["file_name"] = os.path.join(image_dir, cleaned_name)
            updated_images.append(image)
            print(f"Mapped {original_name} to {image['file_name']}")
        else:
            print(f"No match found for {cleaned_name} in {image_dir}")

    coco_data["images"] = updated_images
    updated_json_path = RETRAINED_STAGING_DIR / "results_mapped.json"
    with open(updated_json_path, "w") as f:
        json.dump(coco_data, f, indent=2)
    print(f"Updated COCO JSON saved to {updated_json_path}")
    return updated_json_path

def register_coco_dataset(json_path, image_dir):
    dataset_name = "custom_layout_dataset"
    try:
        if dataset_name in DatasetCatalog:
            DatasetCatalog.remove(dataset_name)
            if dataset_name in MetadataCatalog:
                MetadataCatalog.remove(dataset_name)
            print(f"Unregistered existing dataset: {dataset_name}")

        register_coco_instances(dataset_name, {}, str(json_path), image_dir)
        MetadataCatalog.get(dataset_name).thing_classes = list(LABEL_MAP.values())
        print(f"Registered dataset: {dataset_name}")
    except Exception as e:
        print(f"Error registering dataset: {e}")
        raise
    return dataset_name

def train_layoutparser_model(json_path, image_dir):
    global model
    dataset_name = register_coco_dataset(json_path, image_dir)
    cfg = get_cfg()

    pretrained_model_path = os.path.join(MODELS_DIR, "model_final.pth")
    if os.path.exists(pretrained_model_path):
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = pretrained_model_path
        print(f"Loaded pre-trained weights from {pretrained_model_path} (Faster R-CNN)")
    else:
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
        print("Using default Detectron2 Faster R-CNN model zoo weights")

    cfg.MODEL.DEVICE = str(device)  # "cuda" on T4 GPU
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 2  # GPU-friendly batch size
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 300
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(LABEL_MAP)
    cfg.OUTPUT_DIR = str(RETRAINED_MODELS_DIR)

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Define model_path at function scope
    model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    config_path = os.path.join(cfg.OUTPUT_DIR, "config.yaml")
    with open(config_path, "w") as f:
        f.write(cfg.dump())

    model = lp.Detectron2LayoutModel(
        config_path=config_path,
        model_path=model_path,
        label_map=LABEL_MAP,
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
        device=str(device)
    )
    print(f"Model retrained and saved to {model_path}")

def lazy_load_models():
    global model, ocr_agent
    if ocr_agent is None:
        try:
            ocr_agent = lp.TesseractAgent(languages='eng', config='--psm 6 --oem 1')
            print("OCR agent loaded")
        except Exception as e:
            print(f"Error loading OCR agent: {e}")
            raise
    if model is None:
        model_path = os.path.join(RETRAINED_MODELS_DIR, "model_final.pth")
        config_path = os.path.join(RETRAINED_MODELS_DIR, "config.yaml")
        if os.path.exists(model_path) and os.path.exists(config_path):
            try:
                model = lp.Detectron2LayoutModel(
                    config_path=config_path,
                    model_path=model_path,
                    label_map=LABEL_MAP,
                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                    device=str(device)
                )
                print(f"Loaded retrained model from {model_path}")
            except Exception as e:
                print(f"Error loading model: {e}")
                raise
        else:
            print(f"No retrained model or config found at {model_path} or {config_path}. Training required.")

def setup_db_connection():
    global db_conn, db_cursor, run_number
    if db_conn is None:
        try:
            db_conn = psycopg2.connect(**DB_CONFIG)
            db_cursor = db_conn.cursor()
            print("Connected to Neon PostgreSQL database")
            insert_batch_query = """
                INSERT INTO batch_run (run_date_time, run_by_user, run_final_status)
                VALUES (%s, %s, 'RUNNING') RETURNING run_number
            """
            db_cursor.execute(insert_batch_query, (run_date_time, logged_in_user))
            run_number = db_cursor.fetchone()[0]
            db_conn.commit()
            print(f"Started new batch run: {run_number}")
        except Exception as e:
            print(f"Failed to connect to Neon PostgreSQL: {e}")
            raise

def update_run_status(status):
    global db_conn, db_cursor, run_number
    if db_conn is not None:
        try:
            update_query = """
                UPDATE batch_run
                SET run_final_status = %s
                WHERE run_number = %s
            """
            db_cursor.execute(update_query, (status, run_number))
            db_conn.commit()
            print(f"Updated batch run {run_number} status to {status}")
        except Exception as e:
            print(f"Failed to update run status: {e}")
            db_conn.rollback()

def is_overlapping(rect1, rect2):
    return not (rect1[2] <= rect2[0] or rect2[2] <= rect1[0] or
                rect1[3] <= rect2[1] or rect2[3] <= rect1[1])

def append_skipped_blocks_to_db(skipped_blocks):
    if not skipped_blocks:
        return

    setup_db_connection()
    try:
        MAX_CONTENT_LENGTH = 1000
        values = [
            (
                sb["run_number"],
                sb["parent_block_type"],
                sb["skipped_block_type"],
                sb["section_name"][:255] if sb["section_name"] else None,
                sb["parent_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["parent_block_content"]) > MAX_CONTENT_LENGTH else sb["parent_block_content"],
                sb["skipped_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["skipped_block_content"]) > MAX_CONTENT_LENGTH else sb["skipped_block_content"],
                sb["parent_block_coordinates"],
                sb["skipped_block_coordinates"]
            )
            for sb in skipped_blocks
        ]
        query = """
            INSERT INTO skipped_block_items (
                run_number, parent_block_type, skipped_block_type, section_name,
                parent_block_content, skipped_block_content,
                parent_block_coordinates, skipped_block_coordinates
            ) VALUES %s
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted {len(skipped_blocks)} skipped blocks into skipped_block_items")
    except Exception as e:
        print(f"Skipped blocks database error: {e}")
        db_conn.rollback()

def process_layout(image_path, original_file_path):
    global id_seq, model
    lazy_load_models()

    if model is None:
        raise ValueError("Model not loaded. Ensure a trained model exists in RETRAINED_MODELS_DIR or run training.")

    try:
        image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        raise

    layout = model.detect(image)
    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])
    results = []
    skipped_blocks = []
    list_table_regions = [(b.type.lower(), list(map(int, b.coordinates)))
                         for b in sorted_layout if b.type.lower() in ["list", "table"]]
    current_section = "No Title"

    list_table_contents = {}
    for block_type, coords in list_table_regions:
        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        content = ocr_agent.detect(cropped) or ""
        if block_type == "list":
            content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
        elif block_type == "table":
            content = '\n'.join('-'.join(col.strip() for col in row.split() if col.strip())
                               for row in content.split('\n') if row.strip()) or "Table Content"
        list_table_contents[tuple(coords)] = content.strip()
        del cropped

    block_number = 1
    for block in sorted_layout:
        coords = list(map(int, block.coordinates))
        block_type = block.type.lower()

        overlapping_region = next((r for r in list_table_regions if is_overlapping(coords, r[1])), None)
        if block_type in ["title", "text"] and overlapping_region:
            parent_type, parent_coords = overlapping_region
            cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
            skipped_content = ocr_agent.detect(cropped) or ""
            if block_type == "title":
                skipped_content = skipped_content or "Untitled"
            print(f"Skipping {block_type} inside {parent_type} at coordinates: {coords}")
            skipped_blocks.append({
                "run_number": run_number,
                "parent_block_type": parent_type,
                "skipped_block_type": block_type,
                "section_name": current_section,
                "parent_block_content": list_table_contents[tuple(parent_coords)],
                "skipped_block_content": skipped_content.strip(),
                "parent_block_coordinates": parent_coords,
                "skipped_block_coordinates": coords
            })
            del cropped
            continue

        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        if block_type in ["list", "table"]:
            content = list_table_contents[tuple(coords)]
        elif block_type == "title":
            content = ocr_agent.detect(cropped) or "Untitled"
            current_section = content.lower()
        else:
            content = ocr_agent.detect(cropped) or ""

        result = {
            "id": id_seq,
            "urs": Path(image_path).stem,
            "section": current_section,
            "type": block_type,
            "content": content.strip(),
            "coordinates": coords,
            "created_at": datetime.now().isoformat(),
            "source_file": str(original_file_path),
            "run_number": run_number,
            "block_number": block_number,
        }
        results.append(result)
        id_seq += 1
        block_number += 1

        x1, y1, x2, y2 = coords
        color = COLORS.get(block_type, (255, 255, 255))
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{block_type} #{block_number-1}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        del cropped

    annotated_path = os.path.join(RETRAINED_ANNOTATED_DIR, f"{Path(image_path).stem}_reannotated.png")
    success = cv2.imwrite(annotated_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    if success:
        print(f"Saved reannotated image: {annotated_path}")
    else:
        print(f"Failed to save reannotated image: {annotated_path}")

    del image
    append_skipped_blocks_to_db(skipped_blocks)
    return results

def append_to_db(data):
    if not data:
        return

    setup_db_connection()
    try:
        values = [
            (d["id"], d["urs"], d["section"], d["type"], d["content"],
             d["coordinates"][0], d["coordinates"][1], d["coordinates"][2], d["coordinates"][3],
             d["created_at"], d["source_file"], d["run_number"], d["block_number"])
            for d in data
        ]
        query = """
            INSERT INTO document_metadata (
                id, urs, section, type, content, coord_x1, coord_y1, coord_x2, coord_y2,
                created_at, source_file, run_number, block_number
            ) VALUES %s
            ON CONFLICT (id) DO UPDATE SET
                urs = EXCLUDED.urs,
                section = EXCLUDED.section,
                type = EXCLUDED.type,
                content = EXCLUDED.content,
                coord_x1 = EXCLUDED.coord_x1,
                coord_y1 = EXCLUDED.coord_y1,
                coord_x2 = EXCLUDED.coord_x2,
                coord_y2 = EXCLUDED.coord_y2,
                created_at = EXCLUDED.created_at,
                source_file = EXCLUDED.source_file,
                run_number = EXCLUDED.run_number,
                block_number = EXCLUDED.block_number
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted/Updated {len(data)} records into Neon PostgreSQL")
    except Exception as e:
        print(f"Database error: {e}")
        db_conn.rollback()

def process_and_reannotate(json_path):
    try:
        updated_json_path = map_and_update_coco_paths(json_path, IMG_STAGING_DIR)
        train_layoutparser_model(updated_json_path, IMG_STAGING_DIR)

        with open(updated_json_path, "r") as f:
            coco_data = json.load(f)

        results = []
        for image_info in coco_data["images"]:
            image_path = image_info["file_name"]
            if os.path.exists(image_path):
                print(f"Reannotating {image_path}")
                results.extend(process_layout(image_path, image_path))
            else:
                print(f"Image not found: {image_path}")

        append_to_db(results)
        update_run_status('SUCCESS')
    except Exception as e:
        print(f"Processing failed: {e}")
        print(traceback.format_exc())
        update_run_status('FAILED')
        raise

def cleanup():
    global db_conn, db_cursor
    if db_conn is not None:
        try:
            db_cursor.execute("SELECT run_final_status FROM batch_run WHERE run_number = %s", (run_number,))
            current_status = db_cursor.fetchone()[0]
            if current_status == 'RUNNING':
                update_run_status('TERMINATED')
        except Exception as e:
            print(f"Error during cleanup status check: {e}")
        finally:
            db_cursor.close()
            db_conn.close()
            print("Database connection closed")

if __name__ == "__main__":
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            print(result.stdout)
        else:
            print("nvidia-smi failed:", result.stderr)
    except Exception as e:
        print(f"nvidia-smi not found or error: {e}")

    try:
        process_and_reannotate(COCO_JSON_PATH)
    except KeyboardInterrupt:
        print("Process interrupted by user")
        update_run_status('TERMINATED')
    except Exception as e:
        print(f"Unexpected error: {e}")
        print(traceback.format_exc())
        update_run_status('FAILED')
    finally:
        cleanup()

Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mon Mar 17 14:06:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |    7382MiB /  15360MiB |      0%      De

  roi_heads.mask_head.mask_fcn1.{bias, weight}
  roi_heads.mask_head.mask_fcn2.{bias, weight}
  roi_heads.mask_head.mask_fcn3.{bias, weight}
  roi_heads.mask_head.mask_fcn4.{bias, weight}
  roi_heads.mask_head.deconv.{bias, weight}
  roi_heads.mask_head.predictor.{bias, weight}


[03/17 14:07:06 d2.engine.train_loop]: Starting training from iteration 0
[03/17 14:07:48 d2.utils.events]:  eta: 0:10:07  iter: 19  total_loss: 1.109  loss_cls: 0.7844  loss_box_reg: 0.1996  loss_rpn_cls: 0.0614  loss_rpn_loc: 0.05679    time: 1.9973  last_time: 2.2545  data_time: 0.1243  last_data_time: 0.0061   lr: 1.6068e-05  max_mem: 5196M
[03/17 14:08:25 d2.utils.events]:  eta: 0:08:59  iter: 39  total_loss: 1.29  loss_cls: 0.844  loss_box_reg: 0.2946  loss_rpn_cls: 0.04218  loss_rpn_loc: 0.08576    time: 1.9170  last_time: 1.3280  data_time: 0.0089  last_data_time: 0.0059   lr: 3.2718e-05  max_mem: 5196M
[03/17 14:09:01 d2.utils.events]:  eta: 0:08:02  iter: 59  total_loss: 0.4428  loss_cls: 0.2552  loss_box_reg: 0.1198  loss_rpn_cls: 0.04206  loss_rpn_loc: 0.01815    time: 1.8734  last_time: 2.2976  data_time: 0.0063  last_data_time: 0.0060   lr: 4.9367e-05  max_mem: 5196M
[03/17 14:09:39 d2.utils.events]:  eta: 0:07:22  iter: 79  total_loss: 0.4373  loss_cls: 0.1808  loss_box_